# Image Recognition 

**Objectives:**
* Practising basic machine learning models including K-means, K-NN, SVM, and Adaboost using scit-learn
* Practising Bag-of-Words model
* Applying machine learning models for image recognition

## 1. Bag-of-Words (BoW) model ##

In FoodImages, there are two folders: Train and Test containing training and test images respectively. Each Train/Test folder contains three sub-folders corresponding to three different food types including Cakes, Pasta, and Pizza. For each food category, there are equal numbers of images (30 images) used for training and testing.
We will build a BoW model for food image recognition based on the training images of the supplied food database. The Dictionary class below is developed to build BoW models using K-means algorithm, the 'learn' method where K-means is used and the create_word_histograms method which constructs word histograms for a given list of images. 

In [1]:
import numpy as np
import cv2 as cv
from sklearn.cluster import KMeans

class Dictionary(object):
    def __init__(self, name, img_filenames, num_words):
        self.name = name #name of your dictionary
        self.img_filenames = img_filenames #list of image filenames
        self.num_words = num_words #the number of words
        
        self.training_data = [] #this is the training data required by the K-Means algorithm
        self.words = [] #list of words, which are the centroids of clusters
        
    def learn(self):
        sift = cv.xfeatures2d.SIFT_create()
            
        num_keypoints = [] #this is used to store the number of keypoints in each image
            
        #load training images and compute SIFT descriptors
        for filename in self.img_filenames:
            img = cv.imread(filename)
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            list_des = sift.detectAndCompute(img_gray, None)[1]
            if list_des is None:
                num_keypoints.append(0)
            else:
                num_keypoints.append(len(list_des))
                for des in list_des:
                    self.training_data.append(des)
                        
            
        #cluster SIFT descriptors using K-means algorithm
        kmeans = KMeans(self.num_words)
        kmeans.fit(self.training_data)
        self.words = kmeans.cluster_centers_
            
        #create word histograms for training images
        training_word_histograms = [] #list of word histograms of all training images
        index = 0
        for i in range(0, len(self.img_filenames)):
            #for each file, create a histogram
            histogram = np.zeros(self.num_words, np.float32)
            #if some keypoints exist
            if num_keypoints[i] > 0:
                for j in range(0, num_keypoints[i]):
                    histogram[kmeans.labels_[j + index]] += 1
                index += num_keypoints[i]
                histogram /= num_keypoints[i]
                training_word_histograms.append(histogram)
                    
        return training_word_histograms
        
    def create_word_histograms(self, img_filenames):
        sift = cv.xfeatures2d.SIFT_create()
        histograms = []
            
        for filename in img_filenames:
            img = cv.imread(filename)
            img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            descriptors = sift.detectAndCompute(img_gray, None)[1]
                
            histogram = np.zeros(self.num_words, np.float32) #word histogram for the input image
                
            if descriptors is not None:
                for des in descriptors:
                    #find the best matching word
                    min_distance = 1111111 #this can be any large number
                    matching_word_ID = -1 #initial matching_word_ID=-1 means no matching
                        
                    for i in range(0, self.num_words):  #search for the best matching word
                        distance = np.linalg.norm(des - self.words[i])
                        if distance < min_distance:
                            min_distance = distance
                            matching_word_ID = i
                                
                    histogram[matching_word_ID] += 1
                        
                histogram /= len(descriptors) #normalise histogram to frequencies
                    
            histograms.append(histogram)
                
        return histograms                   

Before building the BoW model for our food database, we need to prepare training data. The following code creates two lists:
* training_file_names: containing the file names of all training images
* training_food_labels: containing the food labels of all training images, e.g., Cakes have labels as 0, Pasta as 1, and Pizza as 2.

In [2]:
import os

foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
training_file_names = []
training_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Train/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    training_file_names += sub_file_names
    training_food_labels += sub_food_labels
    
print(training_file_names)
print(training_food_labels)   


['FoodImages/Train/Cakes/cake1.png', 'FoodImages/Train/Cakes/cake10.jpg', 'FoodImages/Train/Cakes/cake11.jpg', 'FoodImages/Train/Cakes/cake12.jpg', 'FoodImages/Train/Cakes/cake13.jpg', 'FoodImages/Train/Cakes/cake14.jpg', 'FoodImages/Train/Cakes/cake15.jpg', 'FoodImages/Train/Cakes/cake16.jpg', 'FoodImages/Train/Cakes/cake17.jpg', 'FoodImages/Train/Cakes/cake18.jpg', 'FoodImages/Train/Cakes/cake19.jpg', 'FoodImages/Train/Cakes/cake2.png', 'FoodImages/Train/Cakes/cake20.jpg', 'FoodImages/Train/Cakes/cake21.jpg', 'FoodImages/Train/Cakes/cake22.jpg', 'FoodImages/Train/Cakes/cake23.jpg', 'FoodImages/Train/Cakes/cake24.jpg', 'FoodImages/Train/Cakes/cake25.jpg', 'FoodImages/Train/Cakes/cake26.jpg', 'FoodImages/Train/Cakes/cake27.jpg', 'FoodImages/Train/Cakes/cake28.jpg', 'FoodImages/Train/Cakes/cake29.jpg', 'FoodImages/Train/Cakes/cake3.png', 'FoodImages/Train/Cakes/cake30.jpg', 'FoodImages/Train/Cakes/cake4.jpg', 'FoodImages/Train/Cakes/cake5.jpg', 'FoodImages/Train/Cakes/cake6.jpg', 'FoodI

**We are now ready for building the BoW model for our food recognition problem. Suppose that we want to have 50 words in our dictionary (though this number can vary). We name our dictionary 'food' and define it as:**

In [3]:
num_words = 50
dictionary_name = 'food'
dictionary = Dictionary(dictionary_name, training_file_names, num_words)

**We learn the dictionary, i.e., finding words, by calling the learn method as below. The learn method not only extracts words from a training dataset but also creates the word histograms for all the training images in the training set.**

In [4]:
training_word_histograms = dictionary.learn()

In [5]:
import pickle
#save dictionary
with open('food_dictionary.dic', 'wb') as f:  #'wb' is for binary write
    pickle.dump(dictionary, f)

In [6]:
import pickle #you may not need to import it if this has been done
with open('food_dictionary.dic', 'rb') as f:  #'rb' is for binary read
    dictionary = pickle.load(f)

## 2. k-NN ##

**We will apply k-NN for food image recognition**

In [7]:
num_nearest_neighbours = 5 #number of neighbours

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [8]:
test_file_names = ['FoodImages/Test/Pasta/pasta35.jpg']
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [1]


In [9]:
foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
test_file_names = []
test_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Test/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    test_file_names += sub_file_names
    test_food_labels += sub_food_labels
    
print(test_file_names)
print(test_food_labels)  

['FoodImages/Test/Cakes/cake31.jpg', 'FoodImages/Test/Cakes/cake32.jpg', 'FoodImages/Test/Cakes/cake33.jpg', 'FoodImages/Test/Cakes/cake34.jpg', 'FoodImages/Test/Cakes/cake35.jpg', 'FoodImages/Test/Cakes/cake36.jpg', 'FoodImages/Test/Cakes/cake37.jpg', 'FoodImages/Test/Cakes/cake38.jpg', 'FoodImages/Test/Cakes/cake39.jpg', 'FoodImages/Test/Cakes/cake40.jpg', 'FoodImages/Test/Cakes/cake41.jpg', 'FoodImages/Test/Cakes/cake42.jpg', 'FoodImages/Test/Cakes/cake43.jpg', 'FoodImages/Test/Cakes/cake44.jpg', 'FoodImages/Test/Cakes/cake45.jpg', 'FoodImages/Test/Cakes/cake46.jpg', 'FoodImages/Test/Cakes/cake47.jpg', 'FoodImages/Test/Cakes/cake48.jpg', 'FoodImages/Test/Cakes/cake49.jpg', 'FoodImages/Test/Cakes/cake50.jpg', 'FoodImages/Test/Cakes/cake51.jpg', 'FoodImages/Test/Cakes/cake52.jpg', 'FoodImages/Test/Cakes/cake53.jpg', 'FoodImages/Test/Cakes/cake54.jpg', 'FoodImages/Test/Cakes/cake55.jpg', 'FoodImages/Test/Cakes/cake56.jpg', 'FoodImages/Test/Cakes/cake57.jpg', 'FoodImages/Test/Cakes/cake

In [10]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 2 2 0 0 0 1 0 0 2 2 2 2 1 0 0 2 0 0 2 1 0 1 2 0 2 0 1 1 1 1 2 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 2 2 2 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1]


In [12]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[15  5 10]
 [ 0 28  2]
 [ 0  6 24]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        30
           1       0.72      0.93      0.81        30
           2       0.67      0.80      0.73        30

   micro avg       0.74      0.74      0.74        90
   macro avg       0.79      0.74      0.74        90
weighted avg       0.79      0.74      0.74        90

Accuracy: 0.7444444444444445


### Vary num_nearest_neighbours in the range [10, 15, 20, 25, 30] and measure the corresponding accuracies. ###

In [13]:
num_nearest_neighbours = 10 #number of neighbours

knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [14]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 2 2 0 0 0 1 0 0 1 1 2 0 2 0 0 2 1 0 2 1 0 1 2 0 2 0 2 1 2 1 2 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 2 2 2 1 2 1 2 2 2
 2 2 2 2 2 2 1 2 1 2 2 2 2 1 1 1]


In [15]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[15  6  9]
 [ 0 27  3]
 [ 0  9 21]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        30
           1       0.64      0.90      0.75        30
           2       0.64      0.70      0.67        30

   micro avg       0.70      0.70      0.70        90
   macro avg       0.76      0.70      0.69        90
weighted avg       0.76      0.70      0.69        90

Accuracy: 0.7


In [16]:
num_nearest_neighbours = 15 #number of neighbours

knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=15, p=2,
           weights='uniform')

In [17]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 2 2 0 0 0 1 0 0 1 1 2 0 2 0 0 2 1 0 2 1 0 1 2 0 2 0 2 1 2 1 2 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 2 2 2 1 2 1 2 1 2
 2 2 2 2 2 1 2 2 1 2 2 2 2 1 2 1]


In [18]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[15  6  9]
 [ 0 27  3]
 [ 0  9 21]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        30
           1       0.64      0.90      0.75        30
           2       0.64      0.70      0.67        30

   micro avg       0.70      0.70      0.70        90
   macro avg       0.76      0.70      0.69        90
weighted avg       0.76      0.70      0.69        90

Accuracy: 0.7


In [19]:
num_nearest_neighbours = 20 #number of neighbours

knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='uniform')

In [20]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 2 1 0 0 0 1 0 0 1 1 2 2 1 0 0 2 1 0 2 1 2 2 2 0 2 1 2 1 2 1 2 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 2 2 2 1 2 2 1 2 2
 2 2 2 2 2 1 2 2 1 2 2 2 2 1 1 1]


In [21]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[12  8 10]
 [ 0 27  3]
 [ 0  9 21]]
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        30
           1       0.61      0.90      0.73        30
           2       0.62      0.70      0.66        30

   micro avg       0.67      0.67      0.67        90
   macro avg       0.74      0.67      0.65        90
weighted avg       0.74      0.67      0.65        90

Accuracy: 0.6666666666666666


In [22]:
num_nearest_neighbours = 25 #number of neighbours

knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=25, p=2,
           weights='uniform')

In [23]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 2 1 0 0 0 1 0 0 1 2 2 2 1 0 0 2 1 0 2 1 2 1 2 0 2 1 2 1 2 1 2 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 2 2 2 1 2 2 1 2 2
 1 2 2 2 2 1 2 2 1 2 2 2 2 1 1 1]


In [24]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[12  8 10]
 [ 0 27  3]
 [ 0 10 20]]
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        30
           1       0.60      0.90      0.72        30
           2       0.61      0.67      0.63        30

   micro avg       0.66      0.66      0.66        90
   macro avg       0.74      0.66      0.64        90
weighted avg       0.74      0.66      0.64        90

Accuracy: 0.6555555555555556


In [25]:
num_nearest_neighbours = 30 #number of neighbours

knn = KNeighborsClassifier(n_neighbors = num_nearest_neighbours)
knn.fit(training_word_histograms, training_food_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=30, p=2,
           weights='uniform')

In [26]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = knn.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [2 0 0 2 1 2 0 0 1 1 2 1 2 2 2 1 0 0 2 1 0 2 1 2 1 2 0 2 1 2 1 2 1 2 1 1 1
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 2 2 1 2 2 1 1 2
 1 2 2 2 2 2 2 2 1 2 2 2 2 1 1 1]


In [27]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[ 8  9 13]
 [ 0 25  5]
 [ 0  9 21]]
              precision    recall  f1-score   support

           0       1.00      0.27      0.42        30
           1       0.58      0.83      0.68        30
           2       0.54      0.70      0.61        30

   micro avg       0.60      0.60      0.60        90
   macro avg       0.71      0.60      0.57        90
weighted avg       0.71      0.60      0.57        90

Accuracy: 0.6


#### Now we have trained our algorithm and made some predictions. For classification, some commonly used metrics are ‘confusion matrix’, ‘accuracy score’, ‘precision’, ‘recall’ & ‘F1-score or F-measure’ which can be used using the Scikit-learn ‘metric’ library (accuracy_score, confusion_matrix, classification_report). ####

#### Sensitivity or Recall: The proportion of actual positive cases which are correctly identified. ####

#### F1-Score (or F-Measure) is the Harmonic Mean between precision and recall. The range for F1 Score is [0, 1]. It tells you how precise your classifier is (how many instances it classifies correctly), as well as how robust it is. ####

**We can see that the best value for 'num_nearest_neighbours' in this case is '5', where we found that the accuracy is 74.44% precision is 79%, recall is 74% and F-1 score is 0.74. As we increase the value of 'num_nearest_neighbours', the accuracy decreases.**

## 3. SVM ##

**We will use linear SVM to classify food images**

In [28]:
with open('food_dictionary.dic', 'rb') as f:  #'rb' is for binary read
    dictionary = pickle.load(f)

In [29]:
import os

foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
training_file_names = []
training_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Train/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    training_file_names += sub_file_names
    training_food_labels += sub_food_labels
    
print(training_file_names)
print(training_food_labels)   

['FoodImages/Train/Cakes/cake1.png', 'FoodImages/Train/Cakes/cake10.jpg', 'FoodImages/Train/Cakes/cake11.jpg', 'FoodImages/Train/Cakes/cake12.jpg', 'FoodImages/Train/Cakes/cake13.jpg', 'FoodImages/Train/Cakes/cake14.jpg', 'FoodImages/Train/Cakes/cake15.jpg', 'FoodImages/Train/Cakes/cake16.jpg', 'FoodImages/Train/Cakes/cake17.jpg', 'FoodImages/Train/Cakes/cake18.jpg', 'FoodImages/Train/Cakes/cake19.jpg', 'FoodImages/Train/Cakes/cake2.png', 'FoodImages/Train/Cakes/cake20.jpg', 'FoodImages/Train/Cakes/cake21.jpg', 'FoodImages/Train/Cakes/cake22.jpg', 'FoodImages/Train/Cakes/cake23.jpg', 'FoodImages/Train/Cakes/cake24.jpg', 'FoodImages/Train/Cakes/cake25.jpg', 'FoodImages/Train/Cakes/cake26.jpg', 'FoodImages/Train/Cakes/cake27.jpg', 'FoodImages/Train/Cakes/cake28.jpg', 'FoodImages/Train/Cakes/cake29.jpg', 'FoodImages/Train/Cakes/cake3.png', 'FoodImages/Train/Cakes/cake30.jpg', 'FoodImages/Train/Cakes/cake4.jpg', 'FoodImages/Train/Cakes/cake5.jpg', 'FoodImages/Train/Cakes/cake6.jpg', 'FoodI

In [30]:
from sklearn import svm
svm_classifier = svm.SVC(C = 50, kernel = 'linear') 
svm_classifier.fit(training_word_histograms, training_food_labels)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [31]:
test_file_names = ['FoodImages/Test/Pasta/pasta35.jpg']
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [1]


In [32]:
foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
test_file_names = []
test_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Test/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    test_file_names += sub_file_names
    test_food_labels += sub_food_labels
    
print(test_file_names)
print(test_food_labels)  

['FoodImages/Test/Cakes/cake31.jpg', 'FoodImages/Test/Cakes/cake32.jpg', 'FoodImages/Test/Cakes/cake33.jpg', 'FoodImages/Test/Cakes/cake34.jpg', 'FoodImages/Test/Cakes/cake35.jpg', 'FoodImages/Test/Cakes/cake36.jpg', 'FoodImages/Test/Cakes/cake37.jpg', 'FoodImages/Test/Cakes/cake38.jpg', 'FoodImages/Test/Cakes/cake39.jpg', 'FoodImages/Test/Cakes/cake40.jpg', 'FoodImages/Test/Cakes/cake41.jpg', 'FoodImages/Test/Cakes/cake42.jpg', 'FoodImages/Test/Cakes/cake43.jpg', 'FoodImages/Test/Cakes/cake44.jpg', 'FoodImages/Test/Cakes/cake45.jpg', 'FoodImages/Test/Cakes/cake46.jpg', 'FoodImages/Test/Cakes/cake47.jpg', 'FoodImages/Test/Cakes/cake48.jpg', 'FoodImages/Test/Cakes/cake49.jpg', 'FoodImages/Test/Cakes/cake50.jpg', 'FoodImages/Test/Cakes/cake51.jpg', 'FoodImages/Test/Cakes/cake52.jpg', 'FoodImages/Test/Cakes/cake53.jpg', 'FoodImages/Test/Cakes/cake54.jpg', 'FoodImages/Test/Cakes/cake55.jpg', 'FoodImages/Test/Cakes/cake56.jpg', 'FoodImages/Test/Cakes/cake57.jpg', 'FoodImages/Test/Cakes/cake

In [33]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 2 2 2 1 1 1
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 0 2 2 2 2 2 1 2 2 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1]


In [34]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[27  2  1]
 [ 0 24  6]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.90      0.93        30
           1       0.80      0.80      0.80        30
           2       0.78      0.83      0.81        30

   micro avg       0.84      0.84      0.84        90
   macro avg       0.85      0.84      0.85        90
weighted avg       0.85      0.84      0.85        90

Accuracy: 0.8444444444444444


### Vary C in the range [10, 20, 30, 40, 50] and measure the corresponding accuracies. ###

In [35]:
svm_classifier = svm.SVC(C = 10, kernel = 'linear')
svm_classifier.fit(training_word_histograms, training_food_labels)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [36]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 1 0 0 1 2 0 0 2 0 0 0 0 0 2 0 0 0 2 0 0 0 0 1 2 2 2 1 1 1
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 2 1 0 2 2 2 2 2 1 2 2 1 1 2
 1 2 2 2 2 2 2 2 1 2 2 2 2 1 2 1]


In [37]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[24  2  4]
 [ 0 23  7]
 [ 1  8 21]]
              precision    recall  f1-score   support

           0       0.96      0.80      0.87        30
           1       0.70      0.77      0.73        30
           2       0.66      0.70      0.68        30

   micro avg       0.76      0.76      0.76        90
   macro avg       0.77      0.76      0.76        90
weighted avg       0.77      0.76      0.76        90

Accuracy: 0.7555555555555555


In [38]:
svm_classifier = svm.SVC(C = 20, kernel = 'linear')
svm_classifier.fit(training_word_histograms, training_food_labels)

SVC(C=20, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [39]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 1 2 2 2 1 1 1
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 2 1 0 2 2 2 2 2 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 1]


In [40]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 23  7]
 [ 1  5 24]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.77      0.77      0.77        30
           2       0.73      0.80      0.76        30

   micro avg       0.81      0.81      0.81        90
   macro avg       0.82      0.81      0.81        90
weighted avg       0.82      0.81      0.81        90

Accuracy: 0.8111111111111111


In [41]:
svm_classifier = svm.SVC(C = 30, kernel = 'linear')
svm_classifier.fit(training_word_histograms, training_food_labels)

SVC(C=30, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [42]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 1 2 2 2 1 1 1
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 2 1 0 2 2 2 2 2 1 2 2 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1]


In [43]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 23  7]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.79      0.77      0.78        30
           2       0.74      0.83      0.78        30

   micro avg       0.82      0.82      0.82        90
   macro avg       0.83      0.82      0.82        90
weighted avg       0.83      0.82      0.82        90

Accuracy: 0.8222222222222222


In [44]:
svm_classifier = svm.SVC(C = 40, kernel = 'linear')
svm_classifier.fit(training_word_histograms, training_food_labels)

SVC(C=40, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [45]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = svm_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 1 2 2 2 1 1 1
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 2 1 0 2 2 2 2 2 1 2 2 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1]


In [46]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[26  2  2]
 [ 0 23  7]
 [ 1  4 25]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        30
           1       0.79      0.77      0.78        30
           2       0.74      0.83      0.78        30

   micro avg       0.82      0.82      0.82        90
   macro avg       0.83      0.82      0.82        90
weighted avg       0.83      0.82      0.82        90

Accuracy: 0.8222222222222222


**We can see that the best value for 'C' in this case is '50', where we found that the accuracy is 84.44% precision is 85%, recall is 84% and F-1 score is 0.85. As we increase the value of 'C', the accuracy increases.**

## 4. AdaBoost ##

**We will use AdaBoost for food image recognition**

In [47]:
with open('food_dictionary.dic', 'rb') as f:  #'rb' is for binary read
    dictionary = pickle.load(f)

In [48]:
from sklearn.ensemble import AdaBoostClassifier
adb_classifier = AdaBoostClassifier(n_estimators = 150, #weak classifiers
                 random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=150, random_state=0)

In [49]:
test_file_names = ['FoodImages/Test/Pasta/pasta35.jpg']
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [1]


In [50]:
foods = ['Cakes', 'Pasta', 'Pizza']
path = 'FoodImages/'
test_file_names = []
test_food_labels = []
for i in range(0, len(foods)):
    sub_path = path + 'Test/' + foods[i] + '/'
    sub_file_names = [os.path.join(sub_path, f) for f in os.listdir(sub_path)]
    sub_food_labels = [i] * len(sub_file_names) #create a list of N elements, all are i
    test_file_names += sub_file_names
    test_food_labels += sub_food_labels
    
print(test_file_names)
print(test_food_labels)  

['FoodImages/Test/Cakes/cake31.jpg', 'FoodImages/Test/Cakes/cake32.jpg', 'FoodImages/Test/Cakes/cake33.jpg', 'FoodImages/Test/Cakes/cake34.jpg', 'FoodImages/Test/Cakes/cake35.jpg', 'FoodImages/Test/Cakes/cake36.jpg', 'FoodImages/Test/Cakes/cake37.jpg', 'FoodImages/Test/Cakes/cake38.jpg', 'FoodImages/Test/Cakes/cake39.jpg', 'FoodImages/Test/Cakes/cake40.jpg', 'FoodImages/Test/Cakes/cake41.jpg', 'FoodImages/Test/Cakes/cake42.jpg', 'FoodImages/Test/Cakes/cake43.jpg', 'FoodImages/Test/Cakes/cake44.jpg', 'FoodImages/Test/Cakes/cake45.jpg', 'FoodImages/Test/Cakes/cake46.jpg', 'FoodImages/Test/Cakes/cake47.jpg', 'FoodImages/Test/Cakes/cake48.jpg', 'FoodImages/Test/Cakes/cake49.jpg', 'FoodImages/Test/Cakes/cake50.jpg', 'FoodImages/Test/Cakes/cake51.jpg', 'FoodImages/Test/Cakes/cake52.jpg', 'FoodImages/Test/Cakes/cake53.jpg', 'FoodImages/Test/Cakes/cake54.jpg', 'FoodImages/Test/Cakes/cake55.jpg', 'FoodImages/Test/Cakes/cake56.jpg', 'FoodImages/Test/Cakes/cake57.jpg', 'FoodImages/Test/Cakes/cake

In [51]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 0 0 0 1 1 2 0 2 0 0 0 0 0 2 0 0 0 0 0 2 0 0 1 2 1 1 1 1 2
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 1 1 1 0 2 1 2 1 2 2 2 2 1 2 2
 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2]


In [52]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[24  2  4]
 [ 0 23  7]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.96      0.80      0.87        30
           1       0.74      0.77      0.75        30
           2       0.68      0.77      0.72        30

   micro avg       0.78      0.78      0.78        90
   macro avg       0.79      0.78      0.78        90
weighted avg       0.79      0.78      0.78        90

Accuracy: 0.7777777777777778


### Vary n_estimators in the range [50, 100, 150, 200, 250] and measure the corresponding accuracies. ###

In [53]:
adb_classifier = AdaBoostClassifier(n_estimators = 50, #weak classifiers
                 random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=0)

In [54]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 1 0 0 0 0 0 0 0 1 2 0 0 2 0 0 0 0 0 2 0 0 0 0 2 2 0 0 2 1 1 2 2 1 2
 2 2 2 1 1 2 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2]


In [55]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[23  2  5]
 [ 0 20 10]
 [ 0  6 24]]
              precision    recall  f1-score   support

           0       1.00      0.77      0.87        30
           1       0.71      0.67      0.69        30
           2       0.62      0.80      0.70        30

   micro avg       0.74      0.74      0.74        90
   macro avg       0.78      0.74      0.75        90
weighted avg       0.78      0.74      0.75        90

Accuracy: 0.7444444444444445


In [56]:
adb_classifier = AdaBoostClassifier(n_estimators = 100, #weak classifiers
                 random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=0)

In [57]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 0 0 0 1 1 2 0 2 0 0 0 0 0 2 0 0 0 0 0 2 0 0 2 1 1 1 2 1 2
 2 2 2 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 0 2 1 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2]


In [58]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[24  2  4]
 [ 0 22  8]
 [ 1  8 21]]
              precision    recall  f1-score   support

           0       0.96      0.80      0.87        30
           1       0.69      0.73      0.71        30
           2       0.64      0.70      0.67        30

   micro avg       0.74      0.74      0.74        90
   macro avg       0.76      0.74      0.75        90
weighted avg       0.76      0.74      0.75        90

Accuracy: 0.7444444444444445


In [59]:
adb_classifier = AdaBoostClassifier(n_estimators = 200, #weak classifiers
                 random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=200, random_state=0)

In [60]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 2 0 0 0 0 0 2 0 0 0 0 0 2 0 0 1 2 1 2 1 1 2
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 1 1 1 0 2 1 2 1 2 2 2 2 1 2 2
 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2]


In [61]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[25  2  3]
 [ 0 22  8]
 [ 1  6 23]]
              precision    recall  f1-score   support

           0       0.96      0.83      0.89        30
           1       0.73      0.73      0.73        30
           2       0.68      0.77      0.72        30

   micro avg       0.78      0.78      0.78        90
   macro avg       0.79      0.78      0.78        90
weighted avg       0.79      0.78      0.78        90

Accuracy: 0.7777777777777778


In [62]:
adb_classifier = AdaBoostClassifier(n_estimators = 250, #weak classifiers
                 random_state = 0)
adb_classifier.fit(training_word_histograms, training_food_labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=250, random_state=0)

In [63]:
word_histograms = dictionary.create_word_histograms(test_file_names)

predicted_food_labels = adb_classifier.predict(word_histograms)
print('Food label: ', predicted_food_labels)

Food label:  [0 0 0 0 0 0 0 0 0 0 0 1 1 2 0 2 0 0 0 0 0 2 0 0 0 0 0 2 0 0 1 2 2 1 1 1 2
 1 1 2 1 1 2 1 1 1 1 1 1 1 2 2 1 1 1 1 2 1 1 1 2 1 0 2 1 2 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [64]:
print(confusion_matrix(test_food_labels, predicted_food_labels))
print(classification_report(test_food_labels, predicted_food_labels))
print("Accuracy:", accuracy_score(test_food_labels, predicted_food_labels))

[[24  2  4]
 [ 0 22  8]
 [ 1  3 26]]
              precision    recall  f1-score   support

           0       0.96      0.80      0.87        30
           1       0.81      0.73      0.77        30
           2       0.68      0.87      0.76        30

   micro avg       0.80      0.80      0.80        90
   macro avg       0.82      0.80      0.80        90
weighted avg       0.82      0.80      0.80        90

Accuracy: 0.8


**We can see that the best value for 'n_estimators' in this case is '250', where we found that the accuracy is 80.00% precision is 82%, recall is 80% and F-1 score is 0.80. As we increase the value of 'n_estimators', the accuracy increases.**

**In terms of classification algorithms defined here, SVM performed the best classification with an accuracy of 84.44% precision of 85%, recall of 84% and F-1 score of 0.85 which is very accurate.**